# Chater 14. Graph Algorithms

The chapter is on graphs: their implementations (edge list, adjacency list/adjacency map, adjacency matrix) and algorithms (BFS/DFS traversals, Dijkstra’s algorithm).

## Important Data Structures and Algorithms 

In [14]:
# the main graph representation for the chapter
# simplified it a bit (without Vertex and Edge classes, as described in the book)
# I also modified incident_edges(v) for the simplicity of traversal algorithms
class Graph:

    def __init__(self, directed=False):
        self._outgoing = {}
        self._incoming = {} if directed else self._outgoing

    def is_directed(self):
        return self._outgoing is not self._incoming

    def vertex_count(self):
        return len(self._outgoing)

    def edge_count(self):
        count = 0
        for vertex in list(self._outgoing.values()):
            count += len(vertex)
        return count if self.is_directed() else count // 2

    def vertices(self):
        return list(self._outgoing.keys())

    def edges(self):
        edges = set()
        for vertex in self._outgoing:
            edges.update((list(self._outgoing[vertex].values())))        
        return list(edges)

    def get_edge(self, u, v):
        return self._outgoing[u].get(v)

    def degree(self, v, outgoing = True):
        edges = self._outgoing if outgoing else self._incoming
        return(len(edges[v]))

    def incident_edges(self, v, outgoing = True):
        edges = self._outgoing if outgoing else self._incoming
        return edges[v]

    def insert_vertex(self, v):
        self._outgoing[v] = {}
        self._incoming[v] = {}

# C-14.37
# O(n), I dont see a faster method in their implementation of adjacency map
# the whole data structure should rewritte to support O(deg[v])
    def remove_vertex(self, v):
        self._outgoing.pop(v, None)
        for vertex in list(self._outgoing.values()):
            vertex.pop(v, None)
        if self.is_directed():
            self._incoming.pop(v, None)
            for vertex in list(self._incoming.values()):
                vertex.pop(v, None)

    def insert_edge(self, u, v, x):
        self._outgoing[u][v] = x
        self._incoming[v][u] = x

# C-14.38
# O(1) time complexity
    def remove_edge(self, u, v):
        self._outgoing[u].pop(v, None)
        self._incoming[v].pop(u, None)


# here are my implementations of Depth-first search and Breadth-first search
# I like this version better and it fits my ADT for a graph
# I don't store edge values (as they do in the book) as I see no real value in them so far 
def DFS(g, u, discovered):
    if u not in discovered:
        discovered[u] = None
        for e in list(g.incident_edges(u).keys()):
            DFS(g, e, discovered)

def BFS(g, s, discovered):
    level = [s]
    while len(level) > 0:
        next_level = []
        for u in level:
            for e in list(g.incident_edges(u).keys()):
                if e not in discovered:
                    discovered[e] = None
                    next_level.append(e)
        level = next_level

# my implimentation of Dijkstra’s algorithm, used priority queue from the standard 
# library, has the same time-complexity
import heapq

def shortest_path(g, src):
    paths = {vertex : 0 if vertex == src else float('inf') for vertex in g.vertices()}
    visited = {vertex : False for vertex in g.vertices()}
    q = []
    heapq.heappush(q, (paths[src], src))
    while len(q)>0:
        vertex = heapq.heappop(q)[1]
        visited[vertex] = True
        for v, e in g.incident_edges(vertex).items():
            if paths[vertex] + e < paths[v]:
                paths[v] = paths[vertex] + e
            if not visited[v]:
                heapq.heappush(q, (paths[v], v))
    return paths


## Reinforcement

## R-14.6

Suppose we represent a graph ${G}$ having ${n}$ vertices and ${m}$ edges with the edge list structure. Why, in this case, does the insert vertex method run in ${O(1)}$ time while the remove vertex method runs in ${O(m)}$ time?

In [4]:
# the reason is that to insert an edge we either extend a list (amortized O(1)) or a linked
# list (O(1)). To delede an edge we need to scan through all edges m to find the right one
# so we have O(m) in this case


## R-14.7

Give pseudo-code for performing the operation insert_edge(u,v,x) in ${O(1)}$ time using the adjacency matrix representation.

In [15]:
# I have a full implementation in P-14.74 GraphAdjMatrix class as insert_edge(self, u, v, x) method


## Creativity

## C-14.37

Give a Python implementation of the remove_vertex(v) method for our adjacency map implementation of Section 14.2.5, making sure your implementation works for both directed and undirected graphs. Your method should run in ${O(deg(v))}$ time.

In [5]:
# I'd say I've failed here. I don't see a way how in their data structure it could run in the asked time
# see the method remove_vertex(self, v) in the Graph class at the very top with O(n) complexity


## C-14.38

Give a Python implementation of the remove_edge(e) method for our adjacency map implementation of Section 14.2.5, making sure your implementation works for both directed and undirected graphs. Your method should run in ${O(1)}$ time.

In [6]:
# see the method edge(self, e) in the Graph class at the very top 


## C-14.51

Provide an implementation of the BFS algorithm that uses a FIFO queue, rather than a level-by-level formulation, to manage vertices that have been discovered until the time when their neighbors are considered.

In [8]:
from queue import Queue

def BFS_queue(g, s, discovered):
    q = Queue(maxsize = g.vertex_count())
    q.put(s)
    discovered[s] = None
    while not q.empty():
        v = q.get()
        for u in list(g.incident_edges(u).keys()):
            if u not in discovered:
                discovered[u] = None
                q.put(u)
    return (list(discovered.keys()))


## C-14.59

Design an efficient algorithm for finding a longest directed path from a vertex ${s}$ to a vertex ${t}$ of an acyclic weighted directed graph ${\vec{G}^{\,}}$. Specify the graph representation used and any auxiliary data structures used. Also, analyze the time complexity of your algorithm

In [11]:
# the only change is in the sign of normalization process (and '-inf') and in the heap structure
# python doesn't have a builtin heap with the maximum value at the top, so I just added edges with minus sign
# time-complexity is the same as for the regular Dijkstra's algorithm: O([n+m]logn), where n - number of nodes
# m - number of edges
# the data structres used are maximum heap, list and dictionary

def longest_path(g, src):
    paths = {vertex : 0 if vertex == src else float('-inf') for vertex in g.vertices()}
    visited = {vertex : False for vertex in g.vertices()}
    q = []
    heapq.heappush(q, (paths[src], src))
    while len(q)>0:
        vertex = heapq.heappop(q)[1]
        visited[vertex] = True
        for v, e in g.incident_edges(vertex).items():
            if paths[vertex] + e > paths[v]:
                paths[v] = paths[vertex] + e
            if not visited[v]:
                heapq.heappush(q, (-paths[v], v))
    return paths


## Projects

## P-14.74

Use an adjacency matrix to implement a class supporting a simplified graph ADT that does not include update methods. Your class should include a constructor method that takes two collections - a collection ${V}$ of vertex elements and a collection ${E}$ of pairs of vertex elements—and produces the graph G that these two collections represent.

In [10]:
# I took it one stem forward and written the whole functioning class
# here the code got quite ugly from time to time 
# could not make edges() run in O(m)
# if I change the data structure to achieve O(m) there, other important functions
# suffer in terms of time complexity, so it was kind of a tradeoff

class GraphAdjMatrix:

    def __init__(self, directed=False):
        self._matrix = []
        self._vertices = {}
        self._edges_count = 0
        self._directed = directed

    def is_directed(self):
        return self._directed

    def vertex_count(self): 
        return len(self._vertices)

    def edge_count(self): 
        return self._edges_count

    def vertices(self): 
        return list(self._vertices.keys())

    def edges(self):
        edges = set()
        n = self.vertex_count()
        for i in range(n):
            for j in range(n):
                if self._matrix[i][j] != 0:
                    edges.add(self._matrix[i][j])
        return list(edges)

    def get_edge(self, u, v):
        u_index =  self._vertices[u]
        v_index =  self._vertices[v]
        return self._matrix[u][v]

    def degree(self, v, outgoing=True):
        deg = 0
        v_index = self._vertices[v]
        if outgoing:
            deg = sum(x != 0 for x in self._matrix[v_index])
        else:
            deg = sum(x[v_index] != 0 for x in self._matrix)
        return deg
    
    def incident_edges(self, v, outgoing=True): 
        edges = []
        v_index = self._vertices[v]
        if outgoing:
            edges = [x for x in self._matrix[v_index] if x != 0]
        else:
            edges = [x[v_index] for x in self._matrix if x[v_index] != 0]
        return list(set(edges))

    def insert_vertex(self, v):
        old = self._matrix.copy()
        n = len(old)
        if n == 0:
            self._matrix = [[0]]
        else:
            self._matrix = [[0]*(n+1) for i in range(n+1)]
            for i in range(n+1):
                for j in range(n+1):
                    if i != n and j != n:
                        self._matrix[i][j] = old[i][j]
        self._vertices[v] = n

    def remove_vertex(self, v):
        self._edges_count -= self.degree(v)
        removed_vertex_index = self._vertices[v]
        self._change_vertex_list(v)
        n = len(self._matrix)
        new = []*(n-1)*(n-1)
        for i in range(n):
            for j in range(n):
                if not (i == removed_vertex_index or j == removed_vertex_index):
                    new.append(self._matrix[i][j])
        self._matrix = [[0]*(n-1) for i in range(n-1)]
        k = 0
        for i in range(n-1):
            for j in range(n-1):
                self._matrix[i][j] = new[k]
                k+= 1
                
# R-14.7
        
    def insert_edge(self, u, v, x): 
        self._change_edge(u, v, x)
        self._edges_count += 1

    def remove_edge(self, u, v): 
        self._change_edge(u, v, 0)
        self._edges_count -= 1

    def _change_vertex_list(self, v):
        self._vertices.pop(v)
        i = 0
        for vertex in self._vertices:
            self._vertices[vertex] = i
            i += 1
      
    def _change_edge(self, u, v, x):
        u_index = self._vertices[u]
        v_index = self._vertices[v]
        self._matrix[u_index][v_index] = x
        if not self.is_directed():
            self._matrix[v_index][u_index] = x
            

## P-14.75

Implement the simplified graph ADT described in Project P-14.74, using the edge list structure.

In [3]:
# the same point as in the P-14.74: I decided to give a complete implementation
# note that linked list could've been used for better average time-complexity (it removes amortization issue) 
class GraphEdgeList:

    def __init__(self, directed=False):
        self._vertices = []
        self._edges = []
        self._directed = directed

    def is_directed(self): 
        return self._directed

    def vertex_count(self): 
        return len(self._vertices)

    def edge_count(self): 
        return len(self._edges)

    def vertices(self): 
        return self._vertices

    def edges(self): # assuming edges are distinct
        edges = set()
        for edge in self._edges:
            edges.add(edge[2])
        return list(edges)

    def get_edge(self, u, v): 
        for edge in self._edges:
            if self.is_directed():
                if edge[0] == u and edge[1] == v:
                    return edge[2]
            else:
                if u in edge and v in edge:
                    return edge[2]
        return
      
    def degree(self, v, outgoing=True): 
        deg = 0
        i = 0 if outgoing else 1
        for edge in self._edges:
            if self.is_directed():
                if edge[i] == v:
                    deg += 1
            else:
                if v in edge:
                    deg += 1
        return deg

    def incident_edges(self, v, outgoing=True):
        edges = []
        i = 0 if outgoing else 1
        for edge in self._edges:
            if self.is_directed():
                if edge[i] == v:
                    edges.append(edge[2])
            else:
                if v in edge:
                    edges.append(edge[2])
        return edges

    def insert_vertex(self, v): 
        self._vertices.append(v)

    def remove_vertex(self, v):
        self._vertices.remove(v)
        self._edges = [x for x in self._edges if v not in x]

    def insert_edge(self, u, v, x): 
        self._edges.append([u, v, x])

# not O(1), haven't figured out how to achieve this
    def remove_edge(self, u, v): 
        if self.is_directed():
            self._edges = [x for x in self._edges if x[0] != u or x[1] != v]
        else:
            self._edges = [x for x in self._edges if u not in x or v not in x]
            